In [350]:
import os

from tensorflow.python.keras.utils.np_utils import to_categorical

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
print(tf.__version__)

2.8.0


In [351]:
"""Initialization of data generator"""
from data_generator import DataGenerator
generator = DataGenerator(test_data=100, train_data=300)

In [355]:
"""Get train data"""
generator.generate_train_data()
train_data = generator.get_train_data()

In [356]:
"""Get test data"""
generator.generate_test_data()
test_data = generator.get_test_data()

In [357]:
"""Get shapes and examples of generated data"""
print(f'Train: {len(train_data)}, Test: {len(test_data)}')
print(f'Train ex.: {train_data[13]}')
print(f'Test ex.: {test_data[50]}')

Train: 300, Test: 100
Train ex.: RepresentedData(number=3, data=[0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0])
Test ex.: RepresentedData(number=0, data=[0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1])


In [358]:
import random
n1 = random.randint(0, len(train_data) - 1)
print(n1)

131


In [359]:
# переделываем целевые значения
# y_train[i] -- это цифра, подлежащая различению, например 3
# Превратим её в массив [0 0 0 1 0 0 0 0 0 0]
# здесь 1 стоит на 3 месте. Это и есть целевые выходы
# нейронов последнего слоя, чтобы именно нейрон с номером 3
# откликался на изображение цифры 3
# Для такого преобразования есть специальная функция  to_categorical
# Такой формат называется вектор One-hot
print("Целевое значение до преобразования:", train_data[n1].number)

Целевое значение до преобразования: 1


In [360]:
example_y = to_categorical(train_data[n1].number, 10)

In [361]:
print("Целевое значение после преобразования:", example_y)


Целевое значение после преобразования: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [362]:
# Создаём модель слоя
class DenseNN(tf.Module):
    # Конструктор. Заполняет начальные значения
    def __init__(self, outputs, activate="relu"):
        super().__init__()
        self.outputs = outputs  # количество выходов = количеству нейронов в слое
        self.activate = activate  # тип активационной функции (по умолчанию relu)
        self.fl_init = False  # становится true после первой инициализации весов

    # Функция для расчёта выходных величин сети
    def __call__(self, x):
        if not self.fl_init:  # если весов ещё нет, создаём их случайными
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name="b")
            """Размер тензора w: (x.shape[-1], self.outputs)
            здесь x.shape[-1] - размер вектора входных данных,
            то есть, сколько входов в первый раз подадим на сеть,
            столько весов к каждому нейрону и сформируется
            следовательно, первый индекс - номер входа нейрона
            второй размер self.outputs совпадает с количеством выходов
            и с количеством нейронов в слое, значит второй
            индекс - номер нейрона
            stddev=0.1 - среднеквадратическое отклонение"""

            # преобразуем w и b в тензоры
            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.fl_init = True
        # матричное вычисление выхода слоя
        y = x @ self.w + self.b

        if self.activate == "relu":
            return tf.nn.relu(y)
        elif self.activate == "softmax":
            return tf.nn.softmax(y)

        return y


In [363]:
# создадим два полносвязных слоя со 128 нейронами и 10
layer_1 = DenseNN(128)
layer_2 = DenseNN(10, activate="softmax")


In [364]:
# Функция расчёта значений на выходе сети
def model_predict(x):
    y = layer_1(x)
    y = layer_2(y)
    return y  # layer_2(layer_1(x))


In [365]:
# Задание функции потерь
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))
# y_true, y_pred – это наборы one-hot векторов размером мини-батча


In [366]:
# определим оптимизатор для градиентного спуска
opt = tf.optimizers.Adam(learning_rate=0.001)

In [367]:
# Готовим обучение
BATCH_SIZE = 32
EPOCHS = 20
TOTAL = len(train_data)
print(f'Data amount in training set: {TOTAL}')

Data amount in training set: 300


In [369]:
from dataclasses import astuple

test_divided = list(zip(*[astuple(elem) for elem in test_data]))
x_test = test_divided[1]
y_test = test_divided[0]
print(len(y_test))

100


In [370]:
train_divided = list(zip(*[astuple(elem) for elem in train_data]))
x_train = train_divided[1]
y_train = train_divided[0]
print(len(y_train))

300


In [371]:
# y_train

In [372]:
y_temp_train = to_categorical(y_train, 10)
y_temp_test = to_categorical(y_test, 10)
print(len(x_train), len(y_temp_train))

300 300


In [373]:
x_train = tf.reshape(tf.cast(x_train, tf.float32), [300, -1])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [100, -1])
# print("Новый размер: ", tf.shape(x_train).numpy())
# tf.data.Dataset - это класс с набором обучающих данных.
# Его можно создать из списков питона методом from_tensor_slices
# можно из файлов dataset = tf.data.TextLineDataset(["file1.txt", "file2.txt"])
# подробнее https://www.tensorflow.org/api_docs/python/tf/data/Dataset
print(len(x_train), len(y_train))
train_dataset = tf.data.Dataset.from_tensor_slices((list(x_train), list(y_temp_train)))
# [print(i) for i in train_dataset]
# Перемешиваем выборки и разбиваем на батчи
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

300 300


In [374]:
# for x_batch, y_batch in train_dataset:
#     print(x_batch, y_batch)

In [375]:
# Цикл обучения
for n in range(EPOCHS):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))

        loss += f_loss
        grads = tape.gradient(f_loss, [layer_1.trainable_variables, layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        opt.apply_gradients(zip(grads[1], layer_2.trainable_variables))

    print(f'Loss by epochs: {loss.numpy()}')

Loss by epochs: 21.0655517578125
Loss by epochs: 17.094057083129883
Loss by epochs: 12.68157958984375
Loss by epochs: 8.395806312561035
Loss by epochs: 5.001482963562012
Loss by epochs: 2.82195782661438
Loss by epochs: 1.6064165830612183
Loss by epochs: 0.9854227304458618
Loss by epochs: 0.6622207164764404
Loss by epochs: 0.4826820492744446
Loss by epochs: 0.37018537521362305
Loss by epochs: 0.3013482093811035
Loss by epochs: 0.2493566870689392
Loss by epochs: 0.21278265118598938
Loss by epochs: 0.18484452366828918
Loss by epochs: 0.16170856356620789
Loss by epochs: 0.14332939684391022
Loss by epochs: 0.12853722274303436
Loss by epochs: 0.11482445895671844
Loss by epochs: 0.1047537624835968


In [376]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()

In [378]:
# print(y2, y_temp_test)
acc = len(y_temp_test[y_test == y2]) / len(y_temp_test) * 100
print(f'Summary accuracy: {acc}')

Summary accuracy: 60.0
